# Create `cdc-flusight-ensemble` Project on Zoltar
This notebook creates the project template for the `cdc-flusight` Zoltar projects programatically. 

**No knoweledge of json formatting is required.** To edit this template, simply update the variables to correspond with your project.

## Upload YAML to Python Dictionary
update the `cdc-project.yaml` for the CDC Flu Forecasting project. This template shoudn't need much updating, but is a good starting point for future Zoltar projects.

In [10]:
import yaml
import json
import numpy as np
import pymmwr as pm
import datetime
from collections import OrderedDict

template = open("cdc-project.yaml", 'r')
template = yaml.load(template)
print(json.dumps(template, indent=1)[:100])

[
 {
  "name": "CDC Retrospective Forecasts",
  "is_public": true,
  "description": "Guidelines and 


## Replace `lwr` with Correct Bins
This script converts the following target `lwr` inputs from the `cdc-project.yaml` to bin values:
```
lwr: 
    - start: 0
    - end: 13
    - step: 0.1
```

In [11]:
targets = template[0]['targets']
for i in range(len(targets)):
    if 'lwr' in targets[i].keys():
        # update the target lwr
        lwr = targets[i]['lwr']
        
        # extract bin values from lwr
        start = lwr[0]['start']
        end = lwr[1]['end']
        step = lwr[2]['step']
        
        # calculate bin values
        bins = list(np.arange(start, end + step, step).round(2))
        
        # update dictionary with new bins
        targets[i]['lwr'] = bins

## Create `timezeros`

In [12]:
timezeros = template[0]['timezeros']
print(timezeros)

[{'type': 'retrospective'}, {'start': 2010}, {'end': 2017}]


In [13]:
def create_timezeros(timezero_type, start_year, end_year):
    fall = list(range(40,54))
    spring = list(range(1, 21))
    season = fall + spring

    data_version_date = None

    timezeros = []

    for year in range(start_year, end_year):
        for epi_week in season:

            # create timezero_date
            if  40 <= epi_week <= 53: # fall
                ew = pm.Epiweek(year, epi_week)
            else: # spring
                ew = pm.Epiweek(year + 1, epi_week)

            timezero = pm.epiweek_to_date(ew)
            timezero = timezero + datetime.timedelta(days = 1)

            # create data_version_date
            if forecast_type == 'retrospective':
                data_version_date = timezero + datetime.timedelta(weeks = 2)
                data_version_date = data_version_date.strftime('%Y%m%d')

            # format dates
            timezero = timezero.strftime('%Y%m%d')

            # determine is_season_start
            if epi_week == 40:
                is_season_start = True
            else:
                is_season_start = False

            # create season_name
            season_name = str(year) + '-' + str(year + 1)[-2:]

            # create dictionary
            date =  {
              "timezero_date": timezero,
              "data_version_date": data_version_date,
              "is_season_start": is_season_start,
              "season_name": season_name
            }

            timezeros += [date]

    return timezeros

In [14]:
forecast_type = template[0]['timezeros'][0]["type"]
start_year = template[0]['timezeros'][1]["start"]
end_year = template[0]['timezeros'][2]["end"]

timezeros = create_timezeros(forecast_type, start_year, end_year)

## Update template with new Timezeros

In [15]:
template[0]['timezeros'] = timezeros

## Create `cdc-project.json` File

In [16]:
with open('cdc-project-new.json', 'w') as json_file:
    json.dump(template[0], json_file, indent=2)

 ## Create New Project in Zoltar

In [ ]:
from zoltpy import util
conn = util.authenticate()
util.create_project(conn, 'cdc-project.json')